In [1]:
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, GRU, Dense
from tensorflow.keras.optimizers import Adam

file_path = '/kaggle/input/freezing-of-gait/FOG/freezing-of-gait-exploration-main/freezing-of-gait-exploration-main/sliding_window_dataset'
file_names = [
    'S01R01experiment0', 'S01R02experiment0',
    'S02R01experiment0', 'S02R02experiment0',
    'S03R01experiment0', 'S03R01experiment1', 'S03R02experiment0', 'S03R03experiment0',
    'S04R01experiment0',
    'S05R01experiment0', 'S05R01experiment1', 'S05R01experiment2', 'S05R01experiment3',
    'S06R01experiment0',
]
file_name = [
    'S07R01experiment0', 'S07R02experiment0',
    'S08R01experiment0', 'S08R01experiment1', 'S08R01experiment2', 'S08R01experiment3',
    'S09R01experiment0', 'S09R01experiment1'
]


def create_model():
    # Define your model architecture
    model = Sequential()
    model.add(InputLayer((1000, 4)))
    model.add(GRU(64))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

def train_model(X_train, y_train, X_test, y_test, initial_model=None):
    if initial_model is None:
        model = create_model()
    else:
        model = initial_model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=256)

    return model

# Load and preprocess your data
initial_model = None
for name in file_names:
    X = np.load(f'{file_path}/{name}_X.npy')
    y = np.load(f'{file_path}/{name}_y.npy')
    y = np.array([[i] for i in y])
    length = X.shape[0]
    test_data_idx_start = int(length * 0.8)
    X_train = X[:test_data_idx_start]
    y_train = y[:test_data_idx_start]
    X_test = X[test_data_idx_start:]
    y_test = y[test_data_idx_start:]
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

    # Train the model within the TPU strategy scope
    initial_model = train_model(X_train, y_train, X_test, y_test, initial_model)

# Save the final trained model
initial_model.save('/kaggle/working/gru_model.h5')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

(14560, 1000, 4) (14560, 1) (3640, 1000, 4) (3640, 1)
Epoch 1/200
57/57 [==============================] - 12s 69ms/step - loss: 1.0873 - accuracy: 0.1789 - val_loss: 0.8836 - val_accuracy: 0.3942
Epoch 2/200
57/57 [==============================] - 3s 47ms/step - loss: 0.6866 - accuracy: 0.4837 - val_loss: 0.5064 - val_accuracy: 0.7783
Epoch 3/200
57/57 [==============================] - 3s 47ms/step - loss: 0.4678 - accuracy: 0.6897 - val_loss: 0.3214 - val_accuracy: 0.9451
Epoch 4/200
57/57 [==============================] - 3s 49ms/step - loss: 0.3680 - accuracy: 0.8268 - val_loss: 0.2360 - val_accuracy: 0.9695
Epoch 5/200
57/57 [==============================] - 3s 48ms/step - loss: 0.2916 - accuracy: 0.9817 - val_loss: 0.1771 - val_accuracy: 0.9849
Epoch 6/200
57/57 [==============================] - 3s 48ms/step - loss: 0.2249 - accuracy: 0.9959 - val_loss: 0.1378 - val_accuracy: 0.9907
Epoch 7/200
57/57 [==============================] - 3s 53ms/step - loss: 0.1741 - accuracy: 

In [2]:
for name in file_name:
    X1 = np.load(f'{file_path}/{name}_X.npy')
    length = X1.shape[0]
    y1 = np.load(f'{file_path}/{name}_y.npy')
    y1 = np.array([[i] for i in y1])
    test_data_idx_start = int(length)
    X1_test = X1[:test_data_idx_start]
    y1_test = y1[:test_data_idx_start]

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn import metrics

# Load the saved model
model = load_model('/kaggle/working/gru_model.h5')

yhat = model.predict(X1_test)# metrics# metrics
from sklearn import metrics
TN, FP, FN, TP = metrics.confusion_matrix(y1_test, np.array([0 if y < .5 else 1 for y in yhat])).ravel()


print('Sensitivity, hit rate, recall, or true positive rate: ')
TPR = TP/(TP+FN)
print(TPR)
print('Specificity or true negative rate')
TNR = TN/(TN+FP) 
print(TNR)
print('Precision or positive predictive value')
PPV = TP/(TP+FP)
print(PPV)
print('Negative predictive value')
NPV = TN/(TN+FN)
print(NPV)
print('Fall out or false positive rate')
FPR = FP/(FP+TN)
print(FPR)
print('False negative rate')
FNR = FN/(TP+FN)
print(FNR)
print('False discovery rate')
FDR = FP/(TP+FP)
print(FDR)

print('Overall accuracy')
ACC = (TP+TN)/(TP+FP+FN+TN)
print(ACC)

578/578 [==============================] - 10s 16ms/step
Sensitivity, hit rate, recall, or true positive rate: 
0.9896761133603239
Specificity or true negative rate
0.9990398818316101
Precision or positive predictive value
0.9973480212158303
Negative predictive value
0.9962439239946973
Fall out or false positive rate
0.0009601181683899557
False negative rate
0.010323886639676113
False discovery rate
0.0026519787841697267
Overall accuracy
0.9965367965367965
